# Imports

In [1]:
# Please update the default location as you deem fit
cvml_path = '/home/alex.li/git/JupiterCVML/europa/base/src/europa'

In [2]:
import json
import os
import multiprocessing as mp

import numpy as np
import pandas as pd

from cv.core.image_quality_server_side import ImageQuality
from dl.utils.config import DEFAULT_TONEMAP_PARAMS

## Datasets

In [3]:
dataset_idx = 0
directory = ['/data/jupiter/datasets/bad_iq_halo_labelbox_plus_exposure', '/data2/jupiter/datasets/20231017_halo_rgb_labeled_excluded_bad_iq', '/data/jupiter/datasets/iq_2023_v5_anno'][dataset_idx]
csv_name = ['654a5bb2e89875bddc714dd2_master_annotations.csv', '653a7a0a3c2d8ab221f6d915_master_annotations.csv','64dfcc1de5a41169c7deb205_master_annotations.csv'][dataset_idx]

dset_name = directory.split('/')[-1]
save_path='/mnt/sandbox1/alex.li/iq_results'

side_left_tire_mask = f'{cvml_path}/cv/core/tire_masks/side_left_iq_mask.png'
side_right_tire_mask = f'{cvml_path}/cv/core/tire_masks/side_right_iq_mask.png'

# Prediction from JupiterCVML repo

In [4]:
iq = ImageQuality(num_workers=mp.cpu_count() // 2,
                  use_progress=True,
                  side_left_tire_mask_path = side_left_tire_mask,
                  side_right_tire_mask_path = side_right_tire_mask,
                  normalization_params=DEFAULT_TONEMAP_PARAMS,
                  dataset=dset_name,
                  save_path=save_path)
print(directory)
# stereo_df = pd.read_csv(os.path.join(directory, 'master_annotation_with_histogram.csv'), low_memory=False)
stereo_df = pd.read_csv(os.path.join(directory, csv_name), low_memory=False)

/data/jupiter/datasets/bad_iq_halo_labelbox_plus_exposure


In [5]:
stereo_df = stereo_df.drop_duplicates(['id'])

In [6]:
out = pd.value_counts(stereo_df['camera_location'])
for i, v in zip(out.index, out.values):
    print(f"{i},{v}")

T10,2870
T09,1892
T02,1714
T01,994
T05,552
I01,498
T14,405
T13,280
I02,189
T06,141


In [17]:
output_path = f'{save_path}/{dset_name}'
output_file = f'{output_path}/iq.csv'
if not os.path.exists(output_file):
    labeled = iq.from_df(stereo_df, directory, use_progress=True)
else:
    df = pd.read_csv(output_file)

/home/alex.li/miniconda3/envs/cvml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (23,24,25,26,27,29,30,32,33,34,35,36,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
if not os.path.exists(output_file):
    labeled['iq'] = labeled.image_quality.apply(lambda x: x.algorithm_output)
    labeled['iq_features'] = labeled.image_quality.apply(lambda x: x.algorithm_features)
    labeled['iq_features_total'] = labeled.iq_features.apply(lambda x: x['image_features']['total'])
    labeled['iq_features_low'] = labeled.iq_features.apply(lambda x: x['image_features']['low'])
    labeled['iq_features_mid'] = labeled.iq_features.apply(lambda x: x['image_features']['mid'])
    labeled['iq_features_high'] = labeled.iq_features.apply(lambda x: x['image_features']['high'])
    labeled['iq_features_depth_ratio'] = labeled.iq_features.apply(lambda x: x['image_features']['depth_ratio'])
    labeled['iq_features_smudge'] = labeled.iq_features.apply(lambda x: x['image_features']['smudge'])
    labeled['iq_features_smudge_reason'] = labeled.iq_features.apply(lambda x: x['image_features']['smudge_reason'])
    labeled['iq_process_time'] = labeled.image_quality.apply(lambda x: x.algorithm_process_time)

    if 'iq_ground_truth' in labeled:
        labeled['binary_iq'] = labeled.iq.apply(lambda x: 'iq' if x != 'good' else 'non_iq')
        labeled['binary_iq_ground_truth'] = labeled.iq_ground_truth.apply(lambda x: 'iq' if x != 'good' else 'non_iq')

    output_path = f'{save_path}/{dset_name}'
    output_file = f'{output_path}/iq.csv'
    os.makedirs(name=output_path, exist_ok=True)
    labeled.to_csv(output_file)

    labeled.iq.value_counts()
    df = pd.read_csv(f"{output_path}/iq.csv")

In [14]:
for k in df.iq.value_counts().keys():
    print(k)
    print(','.join(sorted(list(set(df[(df.iq == k)]['id'])))))

good
6441ff440c5bcfd4f0c73d80,644200bff9706c0922456463,64420291fb7e8d35a3daa280,644210d5e80424019283e6c7,644211ed9483e5fbaaa8314a,64421813c167bcf5bda98d82,6442192cbef8363a7c354580,64422658242be814b7ae67e7,64422866e80424019284087e,64422e8b28d7da9cfdfade91,6442333b0b6121f98b08486a,64ae6a4b300726b19422019d,64ae72265ff7a72a1cad11ab,64ae733b2267229a9b255c0e,64ae759a9cec289c2f21bd12,64ae79192267229a9b256d05,64ae7f3451621889c928d71d,64ae894151621889c928f97d,64ae894d42a627809b8f8d31,64ae899596da59a3f4bcd8c7,64aea112fe7d8883e38d7c39,64aea21a51621889c9294642,64aea62196da59a3f4bd2f83,64aea6532ffbb8c9feb3a5a7,64aeb365235017ddd0e08d64,64aeb870fe7d8883e38dc276,64aeb8842267229a9b261a1d,64aebd47235017ddd0e0a2e8,64aebdcb235017ddd0e0a473,64aebfbdbe3ada6dacb42bba,64aecf74edd2cf97d08831c3,64aed9755ff7a72a1cae2e73,64aed9952267229a9b267706,64aedac19cec289c2f2280c7,64aedca3edd2cf97d0884c2e,64aedd02c06ead6d4ad1ad5b,64aede22c06ead6d4ad1b13a,64aedf379cec289c2f228a62,64aee08abf952f152b47965b,64aee091e0a80bb5d57b

In [27]:
df.iloc[0]['id']

'64423b34c6ad9344dcaa53e0'

In [30]:
directory + "/processed/images/" + df.iloc[0]['id']

'/data/jupiter/datasets/bad_iq_halo_labelbox_plus_exposure/processed/images/64423b34c6ad9344dcaa53e0'

In [33]:
import matplotlib.pyplot as plt
folder_path  = directory + "/processed/images/" + df.iloc[0]['id'] + "/"
file_name = os.listdir(folder_path)[0]
obj = np.load(folder_path + file_name)

In [35]:
obj['left']

array([[[3460., 3366., 2533.],
        [3480., 3358., 2546.],
        [3464., 3370., 2566.],
        ...,
        [3669., 3682., 2910.],
        [3660., 3648., 2909.],
        [3578., 3618., 2857.]],

       [[3618., 3477., 2650.],
        [3631., 3464., 2646.],
        [3628., 3494., 2676.],
        ...,
        [3702., 3679., 2879.],
        [3676., 3672., 2886.],
        [3695., 3716., 2927.]],

       [[3715., 3572., 2721.],
        [3662., 3496., 2681.],
        [3694., 3560., 2739.],
        ...,
        [3632., 3627., 2823.],
        [3635., 3646., 2862.],
        [3635., 3674., 2877.]],

       ...,

       [[1797., 1978., 1312.],
        [1744., 1971., 1282.],
        [1743., 2023., 1309.],
        ...,
        [3878., 2925., 2155.],
        [3596., 2865., 2090.],
        [3467., 2739., 2001.]],

       [[1765., 1966., 1319.],
        [1696., 1938., 1284.],
        [1683., 1917., 1261.],
        ...,
        [4763., 3073., 2224.],
        [3989., 2618., 1817.],
        [3690.,

## Accuracy Metrics
#### Valid only if ground-truth image_quality_labels exist

In [16]:
def accuracy(df, iq_type='overall'):
    if iq_type!='overall':
        sub_df = df[df.iq_ground_truth==iq_type]
    else:
        sub_df = df
    valid = np.sum(sub_df.iq == sub_df.iq_ground_truth)
    total = len(sub_df)
    acc = valid/total if total != 0 else 0
    print(f'{iq_type}\tproperly-predicted:{valid:4d} \ttotal-images:{total:4d}\tacc:{100*acc:.3f}%')

if 'iq_ground_truth' in df:
    accuracy(df, 'smudge')  
    accuracy(df, 'dark')  
    accuracy(df, 'bright')  
    accuracy(df, 'good') 
    accuracy(df, 'overall')
    valid = np.sum(df.binary_iq == df.binary_iq_ground_truth)
    total = len(df)
    acc = valid/total
    print(f'IqNonIq\tproperly-predicted:{valid:4d} \ttotal-images:{total:4d}\tacc:{100*acc:.3f}%')